In [6]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from math import sqrt 

In [7]:
from prophet import Prophet


c:\Users\Yibabe\Desktop\10academy_kifiya-AIMweek-11\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [8]:
df =pd.read_csv(r'C:\Users\Yibabe\Desktop\10academy_kifiya-AIMweek-11\data\cleaned_data.csv')

In [9]:
df.head()

,Date,Adj Close BND,Adj Close SPY,Adj Close TSLA,Close BND,Close SPY,Close TSLA,High BND,High SPY,High TSLA,Low BND,Low SPY,Low TSLA,Open BND,Open SPY,Open TSLA,Volume BND,Volume SPY,Volume TSLA,Daily_Return
0,2015-01-02 00:00:00+00:00,63.358757,173.173737,14.620667,82.650002,205.429993,14.620667,82.690002,206.880005,14.883333,82.419998,204.179993,14.217333,82.430000,206.380005,14.858000,2218800,121465900,71466000,NaN
1,2015-01-05 00:00:00+00:00,63.542767,170.046280,14.006000,82.889999,201.720001,14.006000,82.919998,204.369995,14.433333,82.699997,201.350006,13.810667,82.739998,204.169998,14.303333,5820100,169632600,80527500,-0.018060
2,2015-01-06 00:00:00+00:00,63.726715,168.444626,14.085333,83.129997,199.820007,14.085333,83.379997,202.720001,14.280000,83.029999,198.860001,13.614000,83.029999,202.089996,14.004000,3887600,209151400,93928500,-0.009419
3,2015-01-07 00:00:00+00:00,63.765030,170.543655,14.063333,83.180000,202.309998,14.063333,83.279999,202.720001,14.318667,83.050003,200.880005,13.985333,83.139999,201.419998,14.223333,2433400,125346700,44526000,0.012461
4,2015-01-08 00:00:00+00:00,63.665379,173.569992,14.041333,83.050003,205.899994,14.041333,83.110001,206.160004,14.253333,82.970001,203.990005,14.000667,83.110001,204.009995,14.187333,1873400,147217800,51637500,0.017745


In [10]:
from sklearn.model_selection import train_test_split

# Split data (assuming your data is indexed by date)
train_size = int(0.8 * len(df))
train, test = df['Close TSLA'][:train_size],df['Close TSLA'][train_size:]

In [11]:
train.shape 

(1979,)

In [12]:
test.shape

(495,)

In [16]:
from pmdarima import auto_arima

# Run auto_arima to find optimal p, d, q values
model_arima = auto_arima(train, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
model_arima.summary() 

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject